# Bounding Box Regression - Four Corner Predict

In [5]:
from ast import literal_eval
from matplotlib import pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [2]:
INIT_LR = 1e-4
NUM_EPOCHS = 25
BATCH_SIZE = 32

own_data_path = 'D:/Github/Spliddit/'

# Preprocess Data
The data is preprocessed in the following manner:
- Loading the labels
- Scaling the labels so that they correspond to the scaled image with 100x100
- Normalizing the label to be between 0 and 1
- Loading and scaling the image

### 1. Loading the labels

In [3]:
#labels_own = pd.read_csv('D:/Github/Spliddit/labels.csv', dtype={'count' : int})
#labels_own['label'] = [literal_eval(x) for x in labels_own.label]

#labels_doc = pd.read_csv('D:/Github/Spliddit/SmartDoc/labels_original.csv', dtype={'count' : int})
#labels_doc['label'] = [literal_eval(x) for x in labels_doc.label]

#labels = labels_own.append(labels_doc).reset_index(drop=True)
#labels = labels.sample(frac=1).reset_index(drop=True)

labels = pd.read_csv('D:/Github/Spliddit/labels_complete.csv', dtype={'count' : int})
labels['label'] = [literal_eval(x) for x in labels.label]

In [4]:
labels.head()

,count,label,path
0,15498,"(630, 256, 1096, 181, 1299, 800, 752, 923)",D:/Github/Spliddit/SmartDoc/SmartDocFrames/001...
1,2776,"(769, 387, 1079, 363, 1210, 756, 857, 809)",D:/Github/Spliddit/SmartDoc/SmartDocFrames/000...
2,8245,"(841, 278, 1198, 255, 1293, 704, 862, 731)",D:/Github/Spliddit/SmartDoc/SmartDocFrames/000...
3,15468,"(651, 209, 1060, 199, 1106, 725, 607, 742)",D:/Github/Spliddit/SmartDoc/SmartDocFrames/001...
4,4638,"(465, 1072, 3687, 977, 3736, 2006, 469, 2044)",D:/Github/Spliddit/dataset/0004638.jpg


### 2. Scaling the labels

In [5]:
def scale_label(height, width, label):
    """Returns the scaled version of the labels so that they
    correspond to the scaled image with a width and height of 100."""
    width_factor = 100/width
    height_factor = 100/height
    label_scaled = []
    for i in range(len(label)):
        if i % 2 == 0:
            label_scaled.append(int(label[i] * width_factor))
        else:
            label_scaled.append(int(label[i] * height_factor))
    return tuple(label_scaled)

In [6]:
x = []
y = []

for i in tqdm(range(labels.shape[0])):
    label = labels.iloc[i, 1]
    img_path = labels.iloc[i, -1]
    img = cv2.imread(img_path)
    height, width = img.shape[:2]
    img = cv2.resize(img, (100,100))
    img = img.astype('float32') / 255
    x.append(img)
    labels_scaled = scale_label(height, width, label)
    labels_normalized = tuple([x/100 for x in labels_scaled])
    y.append(labels_normalized)

x_processed = np.array(x)
y_processed = np.array(y)

100%|████████████████████████████████████████████████████████████████████████████| 33593/33593 [58:51<00:00,  9.51it/s]


In [7]:
x_processed.shape

(33593, 100, 100, 3)

In [8]:
y_processed.shape

(33593, 8)

# Splitting the dataset

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x_processed, y_processed, test_size=0.10, random_state=42)

# writing the data to disk for later use
np.save('D:/Github/Spliddit/final_dataset/x_train.npy', x_train)
np.save('D:/Github/Spliddit/final_dataset/x_test.npy', x_test)
np.save('D:/Github/Spliddit/final_dataset/y_train.npy', y_train)
np.save('D:/Github/Spliddit/final_dataset/y_test.npy', y_test)

In [4]:
# alternatively: loading the data

x_train = np.load('D:/Github/Spliddit/final_dataset/x_train.npy')
x_test = np.load('D:/Github/Spliddit/final_dataset/x_test.npy')
y_train = np.load('D:/Github/Spliddit/final_dataset/y_train.npy')
y_test = np.load('D:/Github/Spliddit/final_dataset/y_test.npy')


# Rebuilding the RecursiveCNN-Architecture

In [10]:
import tensorflow as tf

from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

Building the model

In [11]:
model = models.Sequential()
model.add(layers.Conv2D(20, (5, 5), activation='relu', input_shape=(100, 100, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(40, (5, 5), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(40, (5, 5), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(60, (5, 5), activation='relu'))
model.add(layers.Conv2D(60, (5, 5), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(500))
model.add(layers.Dense(500))
model.add(layers.Dense(8))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 96, 96, 20)        1520      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 48, 20)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 40)        20040     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 22, 40)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 18, 18, 40)        40040     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 9, 9, 40)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 5, 60)          6

In [13]:
opt = Adam(lr=INIT_LR)
model.compile(loss="mse", optimizer=opt)
#latest = tf.train.latest_checkpoint('D:/Github/Spliddit/models/own_model/')
#model.load_weights(latest)

Set checkpoints

In [14]:
# Creating model Checkpoints
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "D:/Github/Spliddit/models/four_corner_model/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq='epoch')

In [15]:
history_own = model.fit(
	x_train, y_train,
	validation_data=(x_test, y_test),
	batch_size=BATCH_SIZE,
	epochs=NUM_EPOCHS,
	verbose=1,
    callbacks = [cp_callback])

Epoch 1/25
945/945 [==============================] - 346s 364ms/step - loss: 0.0181 - val_loss: 0.0040

Epoch 00001: saving model to D:/Github/Spliddit/models/four_corner_model\cp-0001.ckpt
Epoch 2/25
945/945 [==============================] - 355s 376ms/step - loss: 0.0036 - val_loss: 0.0026

Epoch 00002: saving model to D:/Github/Spliddit/models/four_corner_model\cp-0002.ckpt
Epoch 3/25
945/945 [==============================] - 348s 369ms/step - loss: 0.0026 - val_loss: 0.0023

Epoch 00003: saving model to D:/Github/Spliddit/models/four_corner_model\cp-0003.ckpt
Epoch 4/25
945/945 [==============================] - 348s 369ms/step - loss: 0.0020 - val_loss: 0.0018

Epoch 00004: saving model to D:/Github/Spliddit/models/four_corner_model\cp-0004.ckpt
Epoch 5/25
945/945 [==============================] - 346s 366ms/step - loss: 0.0016 - val_loss: 0.0015

Epoch 00005: saving model to D:/Github/Spliddit/models/four_corner_model\cp-0005.ckpt
Epoch 6/25
945/945 [=========================

In [16]:
model.save('D:/Github/Spliddit/models/four_corner_model/full_model')

INFO:tensorflow:Assets written to: D:/Github/Spliddit/models/four_corner_model/full_model\assets


# Testing on new receipts

In [4]:
model = tf.keras.models.load_model('D:/Github/Spliddit/models/own_model/new/model')

### 1. Reading new images and making a prediction

In [17]:
test_images = 0
for file in os.listdir('testset/'):
    test_img = cv2.imread('testset/' + file, 1)
    test_img = cv2.resize(test_img, (100, 100))
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    test_img = test_img.astype('float32') / 255
    if type(test_images) == int:
        test_images = test_img
    else:
        test_images = np.vstack((test_images, test_img))

In [18]:
test_images = np.expand_dims(test_images, -1)
test_images = test_images.reshape((-1, 100, 100, 3))

In [19]:
test_images.shape

(7, 100, 100, 3)

In [20]:
y_pred = model.predict(test_images)

In [21]:
y_pred.shape

(7, 8)

### 2. Denormalizing and upscaling the result
Denormalization is achieved by multiplying the width by 600 and the height by 800

In [22]:
y_pred_denormalized = y_pred * 100
y_pred_denormalized = y_pred_denormalized.astype(int)

Upscaling is achieved by dividing the label by its width and height factors. Since we know the resolution of the test images, we set a constant width and height factor

In [23]:
def descale_result(y_pred):
    width_factor = 100/2268
    height_factor = 100/4032
    pred_descaled = []
    for row in y_pred:
        row_descaled = []
        for cell_index in range(len(row)):
            if(cell_index % 2 == 0):
                row_descaled.append(int(row[cell_index] / height_factor))
            else:
                row_descaled.append(int(row[cell_index] / width_factor))
        pred_descaled.append(row_descaled)
    return pred_descaled

y_pred_descaled = descale_result(y_pred_denormalized)

In [24]:
y_pred_descaled

[[887, 340, 2378, 340, 2298, 1769, 806, 1723],
 [967, 181, 3386, 181, 3064, 1950, 1128, 1973],
 [1209, 113, 2378, 113, 2661, 1950, 1451, 1950],
 [1008, 385, 2580, 385, 2701, 1723, 1128, 1746],
 [766, 181, 2419, 158, 2459, 1837, 806, 1837],
 [967, 408, 2822, 385, 2822, 1769, 1048, 1769],
 [725, 249, 3427, 272, 3104, 2109, 887, 2109]]

### 3. Draw result on test image
Reading the test images

In [25]:
to_display = []

for file in os.listdir('testset/'):
    test_img = cv2.imread('testset/' + file, 1)
    to_display.append(test_img)
    #to_display.append(cv2.resize(test_img, (100,100)))

Drawing the bounding box

In [26]:
def display_bounding_box(img, coords):
    temp = get_coord_pairs(coords)
    # alternative scale
    scaled = []
    for i in temp:
        scaled.append((int(i[0] * 2268), int(i[1] * 4032)))
    temp = scaled
    
    print(temp)
    cv2.circle(img, temp[0], 5, (255,0,0), thickness=-1)
    cv2.circle(img, temp[1], 5,(255,0,0), thickness=-1)
    cv2.circle(img, temp[2], 5,(255,0,0), thickness=-1)
    cv2.circle(img, temp[3], 5,(255,0,0), thickness=-1)
    img = cv2.resize(img, (600, 800))
    cv2.imshow("Outline", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
def get_coord_pairs(coords):
    return [(coords[0], coords[1]), (coords[2], coords[3]), (coords[4], coords[5]), (coords[6], coords[7])]

In [41]:
display_bounding_box(to_display[6], y_pred[6])

[(427, 460), (1935, 508), (1746, 3773), (502, 3750)]


# Corner Refiner

In [1]:
from ast import literal_eval
from matplotlib import pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [2]:
INIT_LR = 1e-4
NUM_EPOCHS = 20
BATCH_SIZE = 32

own_data_path = 'D:/Github/Spliddit/'

# Preprocess Data
The data is preprocessed in the following manner:
- Loading the labels
- Scaling the labels so that they correspond to the scaled image with 100x100
- Normalizing the label to be between 0 and 1
- Loading and scaling the image

### 1. Loading the labels

In [3]:
#labels_own = pd.read_csv('D:/Github/Spliddit/labels.csv', dtype={'count' : int})
#labels_own['label'] = [literal_eval(x) for x in labels_own.label]

#labels_doc = pd.read_csv('D:/Github/Spliddit/SmartDoc/labels_original.csv', dtype={'count' : int})
#labels_doc['label'] = [literal_eval(x) for x in labels_doc.label]

#labels = labels_own.append(labels_doc).reset_index(drop=True)
#labels = labels.sample(frac=1).reset_index(drop=True)

labels = pd.read_csv('D:/Github/Spliddit/labels_corner_refiner_zoom.csv', dtype={'count' : int})
labels['label'] = [literal_eval(x) for x in labels.label]

In [4]:
labels.head()

,count,label,path
0,0,"(630, 256)",D:/Github/Spliddit/dataset_refiner/0000000.jpg
1,1,"(233, 181)",D:/Github/Spliddit/dataset_refiner/0000001.jpg
2,2,"(274, 310)",D:/Github/Spliddit/dataset_refiner/0000002.jpg
3,3,"(752, 334)",D:/Github/Spliddit/dataset_refiner/0000003.jpg
4,4,"(769, 387)",D:/Github/Spliddit/dataset_refiner/0000004.jpg


### 2. Scaling the labels

In [12]:
def scale_label(height, width, label):
    """Returns the scaled version of the labels so that they
    correspond to the scaled image with a width and height of 100."""
    width_factor = 100/width
    height_factor = 100/height
    label_scaled = []
    for i in range(len(label)):
        if i % 2 == 0:
            label_scaled.append(int(label[i] * width_factor))
        else:
            label_scaled.append(int(label[i] * height_factor))
    return tuple(label_scaled)

In [7]:
x = []
y = []

for i in tqdm(range(0, int(labels.shape[0] / 2))):
    label = labels.iloc[i, 1]
    img_path = labels.iloc[i, -1]
    img = cv2.imread(img_path)
    height, width = img.shape[:2]
    img = cv2.resize(img, (100,100))
    img = img.astype('float32') / 255
    x.append(img)
    labels_scaled = scale_label(height, width, label)
    labels_normalized = tuple([x/100 for x in labels_scaled])
    y.append(labels_normalized)

#x_processed = np.array(x)
#y_processed = np.array(y)

100%|████████████████████████████████████████████████████████████████████████| 134372/134372 [1:38:33<00:00, 22.72it/s]


# Splitting the data
... into train and test set as well as data chunks because of memory issues

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.10, random_state=42)

In [23]:
for i in range(0, 13):
    print(i, ' / 13')
    start_big = int(i*(len(x_train)/13))
    end_big = int(start_big +(len(x_train)/13))
    np.save('D:/Github/Spliddit/final_dataset/corner_refiner/x_train_' + str(i) + '.npy', np.array(x_train[start_big:end_big]))
    np.save('D:/Github/Spliddit/final_dataset/corner_refiner/y_train_' + str(i) + '.npy', np.array(y_train[start_big:end_big]))
    
    start_small = int(i*(len(x_test)/13))
    end_small = int(start_small +(len(x_test)/13))
    np.save('D:/Github/Spliddit/final_dataset/corner_refiner/x_test_' + str(i) + '.npy', np.array(x_test[start_small:end_small]))
    np.save('D:/Github/Spliddit/final_dataset/corner_refiner/y_test_' + str(i) + '.npy', np.array(y_test[start_small:end_small]))

0  / 13
1  / 13
2  / 13
3  / 13
4  / 13
5  / 13
6  / 13
7  / 13
8  / 13
9  / 13
10  / 13
11  / 13
12  / 13


# 2nd half

In [24]:
import gc
del x
del y
del x_train
del x_test
del y_train
del y_test


In [25]:
gc.collect()

2165

In [13]:
x = []
y = []

for i in tqdm(range(int(labels.shape[0]/2), labels.shape[0])):
    label = labels.iloc[i, 1]
    img_path = labels.iloc[i, -1]
    img = cv2.imread(img_path)
    height, width = img.shape[:2]
    img = cv2.resize(img, (100,100))
    img = img.astype('float32') / 255
    x.append(img)
    labels_scaled = scale_label(height, width, label)
    labels_normalized = tuple([x/100 for x in labels_scaled])
    y.append(labels_normalized)

#x_processed = np.array(x)
#y_processed = np.array(y)

100%|████████████████████████████████████████████████████████████████████████| 134372/134372 [1:01:13<00:00, 36.58it/s]


In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.10, random_state=42)

In [16]:
for i in range(13, 26):
    print(i, ' / 25')
    i_calc = i - 13
    start_big = int(i_calc*(len(x_train)/13))
    end_big = int(start_big +(len(x_train)/13))
    np.save('D:/Github/Spliddit/final_dataset/corner_refiner/x_train_' + str(i) + '.npy', np.array(x_train[start_big:end_big]))
    np.save('D:/Github/Spliddit/final_dataset/corner_refiner/y_train_' + str(i) + '.npy', np.array(y_train[start_big:end_big]))
    
    start_small = int(i_calc*(len(x_test)/13))
    end_small = int(start_small +(len(x_test)/13))
    np.save('D:/Github/Spliddit/final_dataset/corner_refiner/x_test_' + str(i) + '.npy', np.array(x_test[start_small:end_small]))
    np.save('D:/Github/Spliddit/final_dataset/corner_refiner/y_test_' + str(i) + '.npy', np.array(y_test[start_small:end_small]))

13  / 25
14  / 25
15  / 25
16  / 25
17  / 25
18  / 25
19  / 25
20  / 25
21  / 25
22  / 25
23  / 25
24  / 25
25  / 25


# Rebuilding the RecursiveCNN-Architecture

In [3]:
import tensorflow as tf

from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

Building the model

In [4]:
model = models.Sequential()
model.add(layers.Conv2D(10, (5, 5), activation='relu', input_shape=(100, 100, 3)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(10, (5, 5), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(20, (5, 5), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(30, (5, 5), activation='relu'))
model.add(layers.BatchNormalization())
#model.add(layers.Conv2D(40, (5, 5), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(300))
model.add(layers.Dense(2))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 96, 96, 10)        760       
_________________________________________________________________
batch_normalization (BatchNo (None, 96, 96, 10)        40        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 48, 10)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 10)        2510      
_________________________________________________________________
batch_normalization_1 (Batch (None, 44, 44, 10)        40        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 22, 10)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 18, 18, 20)        5

In [6]:
opt = Adam(lr=INIT_LR)
model.compile(loss="mse", optimizer=opt)
latest = tf.train.latest_checkpoint('D:/Github/Spliddit/models/corner_refiner_model/')
model.load_weights(latest)

Set checkpoints

In [7]:
# Creating model Checkpoints
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "D:/Github/Spliddit/models/corner_refiner_model/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


# Create a callback that saves the model's weights every epoch
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq='epoch')

In [8]:
in_ep = 0

for i in range(0, 26):
    print('\n\n***************************')
    print('*********** ' + str(i) + ' ***********')
    print('***************************\n\n')
    
    # loading the data chunk
    x_train = np.load('D:/Github/Spliddit/final_dataset/corner_refiner/x_train_' + str(i) + '.npy')
    x_test = np.load('D:/Github/Spliddit/final_dataset/corner_refiner/x_test_' + str(i) + '.npy')
    y_train = np.load('D:/Github/Spliddit/final_dataset/corner_refiner/y_train_' + str(i) + '.npy')
    y_test = np.load('D:/Github/Spliddit/final_dataset/corner_refiner/y_test_' + str(i) + '.npy')
    
    history_own = model.fit(
	x_train, y_train,
	validation_data=(x_test, y_test),
	batch_size=BATCH_SIZE,
	epochs= 20 + in_ep,
	verbose=1,
    callbacks = [cp_callback],
    initial_epoch = in_ep)
    
    in_ep += 20



***************************
*********** 0 ***********
***************************


Epoch 1/20
291/291 [==============================] - 100s 343ms/step - loss: 0.9585 - val_loss: 0.1101

Epoch 00001: saving model to D:/Github/Spliddit/models/corner_refiner_model\cp-0001.ckpt
Epoch 2/20
291/291 [==============================] - 99s 339ms/step - loss: 0.1081 - val_loss: 0.0591

Epoch 00002: saving model to D:/Github/Spliddit/models/corner_refiner_model\cp-0002.ckpt
Epoch 3/20
291/291 [==============================] - 100s 343ms/step - loss: 0.0614 - val_loss: 0.0562

Epoch 00003: saving model to D:/Github/Spliddit/models/corner_refiner_model\cp-0003.ckpt
Epoch 4/20
291/291 [==============================] - 99s 340ms/step - loss: 0.0501 - val_loss: 0.0597

Epoch 00004: saving model to D:/Github/Spliddit/models/corner_refiner_model\cp-0004.ckpt
Epoch 5/20
291/291 [==============================] - 99s 341ms/step - loss: 0.0438 - val_loss: 0.0550

Epoch 00005: saving model to D:/Gith

ValueError: Expect x to be a non-empty array or dataset.

In [8]:
in_ep = 260

for i in range(13, 26):
    print('\n\n***************************')
    print('*********** ' + str(i) + ' ***********')
    print('***************************\n\n')
    
    # loading the data chunk
    x_train = np.load('D:/Github/Spliddit/final_dataset/corner_refiner/x_train_' + str(i) + '.npy')
    x_test = np.load('D:/Github/Spliddit/final_dataset/corner_refiner/x_test_' + str(i) + '.npy')
    y_train = np.load('D:/Github/Spliddit/final_dataset/corner_refiner/y_train_' + str(i) + '.npy')
    y_test = np.load('D:/Github/Spliddit/final_dataset/corner_refiner/y_test_' + str(i) + '.npy')
    
    history_own = model.fit(
	x_train, y_train,
	validation_data=(x_test, y_test),
	batch_size=BATCH_SIZE,
	epochs= 20 + in_ep,
	verbose=1,
    callbacks = [cp_callback],
    initial_epoch = in_ep)
    
    in_ep += 20



***************************
*********** 13 ***********
***************************


Epoch 261/280
291/291 [==============================] - 101s 343ms/step - loss: 0.0244 - val_loss: 0.0232

Epoch 00261: saving model to D:/Github/Spliddit/models/corner_refiner_model\cp-0261.ckpt
Epoch 262/280
291/291 [==============================] - 99s 339ms/step - loss: 0.0174 - val_loss: 0.0216

Epoch 00262: saving model to D:/Github/Spliddit/models/corner_refiner_model\cp-0262.ckpt
Epoch 263/280
291/291 [==============================] - 99s 341ms/step - loss: 0.0167 - val_loss: 0.0218

Epoch 00263: saving model to D:/Github/Spliddit/models/corner_refiner_model\cp-0263.ckpt
Epoch 264/280
291/291 [==============================] - 99s 339ms/step - loss: 0.0146 - val_loss: 0.0231

Epoch 00264: saving model to D:/Github/Spliddit/models/corner_refiner_model\cp-0264.ckpt
Epoch 265/280
291/291 [==============================] - 101s 348ms/step - loss: 0.0142 - val_loss: 0.0234

Epoch 00265: saving 

KeyboardInterrupt: 

In [9]:
model.save('D:/Github/Spliddit/models/corner_refiner_model/full_model')

INFO:tensorflow:Assets written to: D:/Github/Spliddit/models/corner_refiner_model/full_model\assets


In [3]:
import tensorflow as tf

In [4]:
model = tf.keras.models.load_model('D:/Github/Spliddit/models/corner_refiner_model/full_model')